In [ ]:
# Bibliotecas 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FuncFormatter
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go 

# Importando a base de dados 
data = pd.read_csv('https://raw.githubusercontent.com/owid/COVID-19-data/master/public/data/owid-covid-data.csv')

# Dicionário de Variáveis 
legendas = pd.read_csv('https://raw.githubusercontent.com/owid/COVID-19-data/master/public/data/owid-covid-codebook.csv')
display(legendas) 

#Configuração de Data e Colunas

# Configurações Tabela
pd.set_option("display.max_columns", None) 

# Definir o formato de exibição para números de ponto flutuante
pd.options.display.float_format = '{:.2f}'.format

# Configurar Gráfico
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['figure.dpi'] = 72

# Configurar para não exibir warnings -> https://pypi.org/project/nb-black/
import warnings
warnings.filterwarnings("ignore")

# Definições de cores -> todas estão numa escala de mais escura para mais clara
AZUL1, AZUL2, AZUL3 = '#010821', '#010A2C', '#011048'
AZUL4, AZUL5, AZUL6 = '#021766', '#041E82', '#022094'
AZUL7, AZUL8, AZUL9 = '#0530D9', '#1A359A', '#3053DC'
VERDE1, VERDE2, VERDE3, VERDE4 = '#02560B', '#068312', '#07BB19', '#1FE132'
VERMELHO1, VERMELHO2, VERMELHO3, VERMELHO4, VERMELHO5 = '#DB0527', '#E23652', '#ED8293', '#F4B4BE', '#FBE6E9'
VERDE1, VERDE2 = '#0C8040', '#9ABB59'
LARANJA1 = '#F79747'
AMARELO1, AMARELO2, AMARELO3, AMARELO4, AMARELO5 = '#FFC700', '#FFCC19', '#FFEB51', '#FFE37F', '#FFEEB2'
BRANCO = '#FFFFFF' 

# Volume de dados do DataFrame
print(f'Entradas no DataFrame: {data.shape[0]} linhas no total')
print(f'Variáveis no DataFrame: {data.shape[1]} colunas no total')

# Verificando as primeiras e últimas entradas
display(data.head())
display(data.tail())

# Transformação dos tipos das colunas
data.date = pd.to_datetime(data.date)

# Verificando o tipo das variáveis
data.info() 

# Verificando dados ausentes
dados_ausentes = round((data.isnull().sum() / data.shape[0]).sort_values(ascending=False),2).reset_index()
dados_ausentes 

# Verificação de Linhas com Dados Duplicados
have_duplicate_rows = data.duplicated().any()
print(have_duplicate_rows) 

# Quantidade de Colunas Numéricas e Não-numéricas
numerics = np.number # numerics = ["int16", "int32", "int64", "float16", "float32", "float64"]

numericas = data.select_dtypes(include=numerics)

nao_numericas = data.select_dtypes(exclude=numerics)

print(
    f"Há {numericas.shape[1]} colunas numéricas e {nao_numericas.shape[1]} colunas não-numéricas."
)

# Encontrar as siglas que começam com "OWID" na coluna "iso_code"
siglas_owid = data[data['iso_code'].str.startswith('OWID')]['iso_code'].unique()

# Exibir as siglas encontradas
print("Siglas que começam com 'OWID':")
for i, sigla in enumerate(siglas_owid):
    print(i, sigla)

data_owid = data[data['iso_code'].str.startswith('OWID')]

delete_rows_owid = data_owid.new_deaths > 100000

data_owid = data_owid[delete_rows_owid]

display(data_owid) 

# Criar um novo DataFrame excluindo as linhas com siglas que começam com "OWID"
data_new = data[data['iso_code'].str.startswith('OWID')]
print(data_new.iso_code.unique()) 

# Criar a lista de condições para excluir
delete_rows = [
    (data_new['location'] == 'Germany') & (data_new['date'] == '2020-01-05') & (data_new['new_deaths'] == 3.0),
    (data_new.query('new_deaths > 100000'))
]
condicao_final = pd.concat(delete_rows, axis=1).any(axis=1)

# Exclusão das linhas
data_new = data_new[condicao_final]
print(data_new.shape) 

# Filtrar as linhas com valores não nulos em "new_cases"
cases = data_new.dropna(subset=['new_cases'])

# Filtrar as linhas onde "new_cases" não é igual a zero
cases_clean = cases[cases['new_cases'] != 0.0]

# Ordenar as datas em ordem crescente
cases_clean = cases_clean.sort_values('date')

# Exibir a data mínima do DataFrame filtrado
data_primeiro_caso = cases_clean['date'].min()
print(f"Data do primeiro caso: {data_primeiro_caso}")

# Procurar informações sobre o primeiro caso em 2020-01-04
print("Informações sobre o primeiro caso:")
data_new[(data_new['date'] == '2020-01-04') & (data_new['new_cases'] != 0.0)] 

# Filtrar as linhas com valores não nulos em "new_deaths"
deaths = data_new.dropna(subset=['new_deaths'])

# Filtrar as linhas onde "new_deaths" não é igual a zero
deaths_clean = deaths.loc[deaths['new_deaths'] != 0.0]

# Ordenar as datas em ordem crescente
deaths_clean = deaths_clean.sort_values('date')

# Exibir a data mínima do DataFrame filtrado
data_primeira_morte = deaths_clean['date'].min()
print(f"Data da primeira morte: {data_primeira_morte}")

# Procurar informações sobre a primeira morte em 2020-01-12
print("Informações sobre a primeira morte:")
data_new[(data_new['date'] == '2020-01-12') & (data_new['new_deaths'] != 0.0)]

data_new[["new_cases", "new_deaths"]].describe().T 

# Filtrar dados com mais de 6 milhões de casos no mundo
filtered_data = data_owid.query("iso_code == 'OWID_WRL' and new_cases > 7000000")
display(filtered_data) 

# Filtrar dados para verificar maiores registros de casos
filtered_date = data_new.query("date == '2022-12-23'")
total_new_cases = filtered_date['new_cases'].max()
print("Quantidade com maior registro:", total_new_cases)
query_max = data_new.query("new_cases == 6966046.0 and date == '2022-12-23'")
display(query_max) 

# Filtrar dados com mais de 20 mil mortes diárias no mundo
filtered_data = data_owid.query("iso_code == 'OWID_WRL' and new_deaths > 20000") 

# Filtrar dados com mais de 6 milhões de casos no mundo
filtered_data = data_new.new_deaths.max()
filtered_data = data_new.query('new_deaths > 10000')
display(filtered_data) 

# Criando uma matriz de correlação
data_corr = data.corr()

# Número de variáveis a serem selecionadas
k = 10

# Encontrando as variáveis mais correlacionadas com total_deaths
cols = data_corr.nlargest(k, 'new_deaths')['new_deaths'].index
cm = data[cols].corr()

# Plotando o heatmap
plt.figure(figsize=(10, 8))
hm = sns.heatmap(cm, cmap='Blues', cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 8}, yticklabels=cols.values, xticklabels=cols.values)
plt.title('Maiores Correlações Positivas com Novos Casos de Óbitos', fontsize=18, fontweight='bold')
plt.tight_layout()
plt.show()  

# Agrupamento dos países com mais casos no total
top_countries_cases = data_new.groupby('location')['total_cases'].max().sort_values(ascending=False)[:10]
top_countries_cases = top_countries_cases.reset_index()
top_countries_cases.columns = ['País', 'Total Casos']
# display(top_countries_cases)

# Criar a figura e os eixos
fig, ax = plt.subplots(figsize=(12, 6))

# Criar o gráfico de barras
bar_total_cases = ax.bar(top_countries_cases['País'], top_countries_cases['Total Casos'], color=VERDE1, width=0.7)

# Formatar os números sem notação científica
#ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

# Função para formatar valores em milhões
def millions_formatter(x, pos):
    return f'{x/1e6:.0f}'

# Formatar eixo y para números em milhões
ax.yaxis.set_major_formatter(FuncFormatter(millions_formatter))

# Remover grid
ax.grid(False)

# Remover linhas
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Definir cores das linhas
ax.spines['bottom'].set_color(AZUL3)
ax.spines['left'].set_color(AZUL3)

# Configurar rótulos do eixo y
ax.set_ylabel('Total de Casos', color=VERDE1, fontsize=12, fontweight='bold')

# Definir cor cinza dos valores do eixo x
ax.tick_params(axis='x', colors=VERDE1, labelsize=11)

# Definir cor cinza dos valores do eixo y
ax.tick_params(axis='y', colors=AZUL3, labelsize=12)

# Remover yticks do eixo y
ax.tick_params(bottom=False)

# Formatação dos rótulos de numeral completo para arredondado
def formatar_rotulo(valor):
    if valor >= 1_000_000:
        return f'{valor / 1_000_000:.1f}M'
    else:
        return str(valor)

# Adicionar rótulos de dados e cores destacadas
for i, rect in enumerate(bar_total_cases):
    height = rect.get_height()
    valor_formatado = formatar_rotulo(height)
    if i < 2:  # Destaque para barras dos EUA e China
        rect.set_color(VERDE1)  # Cor destaque
    else:
        rect.set_color(VERDE3)  # Cor padrão para outras barras

    # Rótulos de dados
    ax.text(rect.get_x() + rect.get_width() / 2,
            height / 1.03,
            valor_formatado,
            ha='center',
            va='top',
            color=BRANCO)

# Título
ax.text(-0.065, 1.18,'Países com Mais Casos Registrados',
        fontsize=24,
        color=VERDE3,
        fontweight='bold',
        transform=ax.transAxes)

# Subtítulo
ax.text(-0.065, 1.04,'No período de 2020 a 2023, os países listados abaixo somaram mais de'
                     '480 milhões de casos de COVID-19. \nNotável é o destaque para os Estados Unidos e a China,'
                     'que ocupam a posição de liderança na contagem de casos registrados do vírus..',
        fontsize=14.5,
        color=VERDE4,
        transform=ax.transAxes)

# Legenda
ax.text(0.23, 0.80, 'Esse valor ultrapassa a soma dos registros '
                    'dos cinco países subsequentes \nna lista das nações com mais casos'
                    ' de COVID-19 no mundo.',

        fontsize=12,
        color=VERDE4,
        transform=ax.transAxes)



# Ajustar espaço entre os plots
plt.tight_layout()

# Apresentar o gráfico
plt.show()

# Agrupamento dos países com mais casos por milhão
top_countries_cases_per_million = data_new.groupby('location')['total_cases_per_million'].max().sort_values(ascending=False)[:10]
top_countries_cases_per_million = top_countries_cases_per_million.reset_index()
top_countries_cases_per_million.columns = ['País', 'Total de Casos por Milhão']

# Ordenar o DataFrame em ordem decrescente pelo valor 'Total de Casos por Milhão'
top_countries_cases_per_million = top_countries_cases_per_million.sort_values(by='Total de Casos por Milhão', ascending=True)

# Criar a figura e os eixos
fig, ax = plt.subplots(figsize=(12, 6))

# Criar o gráfico de barras horizontais
bar_million_cases = ax.barh(top_countries_cases_per_million['País'], top_countries_cases_per_million['Total de Casos por Milhão'], color=AZUL1)

# Formatar os números sem notação científica
ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

# Remover grid
ax.grid(False)

# Remover linhas
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Definir cores das linhas
ax.spines['bottom'].set_color(AZUL3)
ax.spines['left'].set_color(AZUL3)

# Configurar rótulos do eixo y
ax.set_xlabel('Total de Casos por Milhão de Habitantes', color=AZUL5, fontsize=12, fontweight='bold')

# Definir cor cinza dos valores do eixo x
ax.tick_params(axis='x', colors=AZUL3, labelsize=11)

# Definir cor cinza dos valores do eixo y
ax.tick_params(axis='y', colors=AZUL3, labelsize=12)

# Remover ticks dos eixo y
ax.tick_params(left=False)

# Formatação dos rótulos de numeral completo para arredondado e milhares
def formatar_rotulo(valor):
    valor_arredondado = round(valor / 1000)
    return f'{valor_arredondado} Mil'

# Adicionar rótulos de dados formatados
for rect in bar_million_cases:
    width = rect.get_width()
    label = formatar_rotulo(width)
    ax.text(rect.get_x() + rect.get_width() / 1.01, rect.get_y() + rect.get_height() / 2,
            label, ha='right', va='center', color='white')

# Título
ax.text(-0.12, 1.18,'Dez países com mais casos por habitantes',
        fontsize=24,
        color=AZUL3,
        fontweight='bold',
        transform=ax.transAxes)

# Subtítulo
ax.text(-0.12, 1.04,'O gráfico apresenta os 10 países com as maiores taxas de'
                      ' casos de COVID-19 por habitantes.\nIsso permite'
                      ' uma análise mais precisa e contextualizada da propagação do coronavírus.',
        fontsize=14.5,
        color=AZUL5,
        transform=ax.transAxes)

# Ajustar espaço entre os plots
plt.tight_layout()

# Apresentar o gráfico
plt.show()

# Filtrar os dados para os novos casos no mundo
world_new_cases = data_owid.loc[data_owid.location == "World", ["date", "new_cases"]]

# Criar a figura e os eixos
fig, ax = plt.subplots(figsize=(16, 6), dpi=72)

# Criar o gráfico de linha
ax.plot(world_new_cases["date"], world_new_cases["new_cases"], color=AZUL1)

# Formatar eixo y para números sem notação científica
ax.yaxis.set_major_formatter(ScalarFormatter(useOffset=False, useMathText=True))
ax.ticklabel_format(style='plain', axis='y')  # Forçar formato de número

# Função para formatar valores em milhões
def millions_formatter(x, pos):
    return f'{x/1e6:.0f}'

# Formatar eixo y para números em milhões
ax.yaxis.set_major_formatter(FuncFormatter(millions_formatter))

# Definir os anos como ticks no eixo x
ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True))
ax.set_xticks(world_new_cases["date"][::365])  # A cada 365 dias (um ano)

# Remover grid
ax.grid(False)

# Remover linhas
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Definir cores das linhas
ax.spines['left'].set_color(AZUL5)
ax.spines['bottom'].set_color(AZUL5)

# Configurar rótulos do eixo y
ax.set_ylabel('Quantidade de Casos por Dia \n (Em Milhões)', color=AZUL5, fontsize=12, fontweight='bold')

# Definir cor cinza dos valores do eixo x
ax.tick_params(axis='x', colors=AZUL5, labelsize=10)

# Definir cor cinza dos valores do eixo y
ax.tick_params(axis='y', colors=AZUL3, labelsize=12)

# Remover yticks do eixo y
ax.tick_params(left=False)

# Título / Subtítulo / Legendas
ax.text(-0.05, 1.1,'Evolução dos Casos de COVID-19 entre 2020 e 2023',
        fontsize=24,
        color=AZUL3,
        fontweight='bold',
        transform=ax.transAxes)

# Subtítulo
ax.text(-0.05, 1.04,'O gráfico apresenta os registros diários dos casos de '
                    'coronavírus ao redor do mundo.',
        fontsize=14.5,
        color=AZUL5,
        transform=ax.transAxes)

# Legendas
ax.text(0.13, 0.78,'A diseminação da variante ômicron, mais transmissível,'
                  'combinada ao \n afrouxamento das medidas de proteção'
                  'como o uso de\nmáscaras e o distanciamento social, '
                  'provavelmente contribuiu para o \naumento dos casos '
                  'registrados no início de 2022 ao redor do globo.',
        fontsize=12,
        color=AZUL4,
        transform=ax.transAxes)

ax.text(0.8, 0.79,'China atingiu pico de mais de '
                    '\n7 milhões de casos diários por '
                    '\nCOVID-19 no final de 2022.',
        fontsize=12,
        color=AZUL4,
        transform=ax.transAxes)

# Ajustar espaço entre os plots
plt.tight_layout()

# Exibir o gráfico
plt.show()


# Filtrar os países com mais mortes por milhão
top_countries_deaths_per_million = data_new.groupby('location')['total_deaths_per_million'].max().sort_values(ascending=False)[:10]
top_countries_deaths_per_million = top_countries_deaths_per_million.reset_index()
top_countries_deaths_per_million.columns = ['País', 'Total de Mortes por Milhão']

# Ordenar o DataFrame em ordem decrescente pelo valor 'Total de Mortes por Habitantes'
top_countries_deaths_per_million = top_countries_deaths_per_million.sort_values(by='Total de Mortes por Milhão', ascending=True)

# Criar a figura e os eixos
fig, ax = plt.subplots(figsize=(12, 6))

# Criar o gráfico de barras
bar_total_deaths = ax.barh(top_countries_deaths_per_million['País'], top_countries_deaths_per_million['Total de Mortes por Milhão'], color=VERMELHO1)

# Formatar os números sem notação científica
ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

# Remover grid
ax.grid(False)

# Remover linhas
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Definir cores das linhas
ax.spines['bottom'].set_color(AZUL3)
ax.spines['left'].set_color(AZUL3)

# Configurar rótulos do eixo y
ax.set_xlabel('Total de Óbitos por Habitantes', color=AZUL5, fontsize=12, fontweight='bold')

# Definir cor cinza dos valores do eixo x
ax.tick_params(axis='x', colors=AZUL3, labelsize=11)

# Definir cor cinza dos valores do eixo y
ax.tick_params(axis='y', colors=AZUL3, labelsize=12)

# Remover ticks dos eixo y
ax.tick_params(left=False)

# Formatação dos rótulos de numeral completo para arredondado e milhares
def formatar_rotulo(valor):
    rotulo = valor / 1000
    return f'{rotulo:.1f} Mil'

# Adicionar rótulos de dados formatados
for rect in bar_total_deaths:
    width = rect.get_width()
    label = formatar_rotulo(width)
    ax.text(rect.get_x() + rect.get_width() / 1.01, rect.get_y() + rect.get_height() / 2,
            label, ha='right', va='center', color='white')

# Título
ax.text(-0.22, 1.18,'Dez países com mais mortes por milhão de habitantes',
        fontsize=24,
        color=AZUL3,
        fontweight='bold',
        transform=ax.transAxes)

# Subtítulo
ax.text(-0.22, 1.04,'O gráfico apresenta os 10 países com as maiores taxas de'
                      ' mortes de COVID-19 por habitantes.\nIsso permite'
                      ' uma análise mais precisa e contextualizada da letalidade do coronavírus.',
        fontsize=14.5,
        color=AZUL5,
        transform=ax.transAxes)

# Ajustar espaço entre os plots
plt.tight_layout()

# Apresentar o gráfico
plt.show()

# Filtrar os dados para as novas mortes no mundo
world_new_deaths = data_owid.loc[data.location == "World", ["date", "new_deaths"]]

# Criar a figura e os eixos
fig, ax = plt.subplots(figsize=(16, 6), dpi=72)

# Criar o gráfico de linha
ax.plot(world_new_deaths["date"], world_new_deaths["new_deaths"], linestyle='-', color=VERMELHO1)

# Definir os anos como ticks no eixo x
ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True))
ax.set_xticks(world_new_cases["date"][::365])  # A cada 365 dias (um ano)

# Formatar os números sem notação científica
ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

# Remover grid
ax.grid(False)

# Remover linhas
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Definir cores das linhas
ax.spines['left'].set_color(AZUL1)
ax.spines['bottom'].set_color(AZUL1)

# Configurar rótulos do eixo y
ax.set_ylabel('Quantidade de Óbitos por Dia', color=AZUL1, fontsize=12, fontweight='bold')

# Definir cor cinza dos valores do eixo x
ax.tick_params(axis='x', colors=AZUL1, labelsize=12)

# Definir cor cinza dos valores do eixo y
ax.tick_params(axis='y', colors=AZUL3, labelsize=12)

# Remover yticks do eixo y
ax.tick_params(left=False)

# Título
ax.text(-0.06, 1.15,'Registro diário de óbitos da pandemia da COVID-19 entre 2020 e 2023',
        fontsize=24,
        color=AZUL3,
        fontweight='bold',
        transform=ax.transAxes)

# Subtítulo
ax.text(-0.06, 1.07,'O maior pico de registro de óbitos pela COVID-19 ao '
                    'redor do mundo foi no final de 2021 e início de 2022.',
        fontsize=14.5,
        color=AZUL4,
        transform=ax.transAxes)

# Ajustar espaço entre os plots
plt.tight_layout()

# Exibir o gráfico
plt.show() 

# Criar uma cópia do DataFrame apenas com o Brasil
data_brazil = data.loc[data.iso_code == "BRA"].copy()

display(data_brazil)

# Identificando o primeiro caso
data_brazil.loc[data_brazil['new_cases'] != 0.0].head(1)

# Identificando a primeira morte
data_brazil.loc[data_brazil['new_deaths'] != 0.0].head(1) 

data_brazil[["new_cases", "new_deaths"]].describe().T 

data_brazil.query('new_cases > 200000')

data_brazil.query('new_deaths> 4000') 

# Agrupar por ano e pegar a última linha de cada grupo
death_cases_year_diff = data_brazil.groupby(data_brazil['date'].dt.year).last()

# Selecionar apenas as colunas de interesse
death_cases_year_diff = death_cases_year_diff[['total_deaths', 'total_cases']]

# Calcular a diferença entre os anos
death_cases_year_diff['diff_total_deaths'] = death_cases_year_diff['total_deaths'].diff()
death_cases_year_diff['diff_total_cases'] = death_cases_year_diff['total_cases'].diff()

# Manter o valor da primeira linha (2020) igual ao total
death_cases_year_diff['diff_total_deaths'].iloc[0] = death_cases_year_diff['total_deaths'].iloc[0]
death_cases_year_diff['diff_total_cases'].iloc[0] = death_cases_year_diff['total_cases'].iloc[0]

# Formatar valores com milhares e sem casas decimais
format_func = lambda x: '{:,.0f}'.format(x).replace(',', '.')

# Aplicar a função de formatação para as colunas de diferença
death_cases_year_diff[['diff_total_deaths', 'diff_total_cases']] = death_cases_year_diff[['diff_total_deaths', 'diff_total_cases']].applymap(format_func)

# Renomear o índice para 'year' e exibir o DataFrame
last_columns = death_cases_year_diff[['diff_total_deaths', 'diff_total_cases']]
last_columns.index.name = 'Ano'
last_columns.columns = ['Mortes por Ano', 'Casos por Ano']
last_columns 

# Criar a figura e os eixos
fig, ax = plt.subplots(figsize=(16, 6), dpi=72)

# Criar o gráfico de linha
ax.plot(data_brazil["date"], data_brazil["new_cases"], linestyle='-', color=AZUL1)

# Definir os anos como ticks no eixo x
ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True))
ax.set_xticks(world_new_cases["date"][::365])  # A cada 365 dias (um ano)

# Formatar os números sem notação científica
ax.get_yaxis().set_major_formatter(plt.FuncFormatter(lambda x, loc: "{:,}".format(int(x))))

# Remover grid
ax.grid(False)

# Remover linhas
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Definir cores das linhas
ax.spines['left'].set_color(AZUL5)
ax.spines['bottom'].set_color(AZUL5)

# Configurar rótulos do eixo y
ax.set_ylabel('Quantidade de Casos por Dia', color=AZUL5, fontsize=12, fontweight='bold')

# Definir cor cinza dos valores do eixo x
ax.tick_params(axis='x', colors=AZUL5, labelsize=12)

# Definir cor cinza dos valores do eixo y
ax.tick_params(axis='y', colors=AZUL3, labelsize=12)

# Remover yticks do eixo y
ax.tick_params(left=False)

# Título
ax.text(-0.07, 1.17,'Evolução de casos de COVID-19 no Brasil entre 2020 e 2023',
        fontsize=24,
        color=AZUL3,
        fontweight='bold',
        transform=ax.transAxes)

# Subtítulo
ax.text(-0.07, 1.04,'Mais de 37 milhões de casos foram registrados no Brasil,'
                    'atingindo um pico de mais \nde 200 mil novos casos '
                    'por dia no início de 2022.',
        fontsize=14.5,
        color=AZUL5,
        transform=ax.transAxes)

# Ajustar espaço entre os plots
plt.tight_layout()

# Exibir o gráfico
plt.show() 

# Criar a figura e os eixos
fig, ax = plt.subplots(figsize=(16, 6), dpi=72)

# Criar o gráfico de linha
ax.plot(data_brazil["date"], data_brazil["new_deaths"], linestyle='-', color=VERMELHO1)

# Definir os anos como ticks no eixo x
ax.xaxis.set_major_locator(plt.MaxNLocator(integer=True))
ax.set_xticks(world_new_cases["date"][::365])  # A cada 365 dias (um ano)

# Remover grid
ax.grid(False)

# Remover linhas
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# Definir cores das linhas
ax.spines['left'].set_color(AZUL5)
ax.spines['bottom'].set_color(AZUL5)

# Configurar rótulos do eixo y
ax.set_ylabel('Quantidade de Óbitos por Dia', color=AZUL5, fontsize=12, fontweight='bold')

# Definir cor cinza dos valores do eixo x
ax.tick_params(axis='x', colors=AZUL5, labelsize=12)

# Definir cor cinza dos valores do eixo y
ax.tick_params(axis='y', colors=AZUL3, labelsize=12)

# Remover yticks do eixo y
ax.tick_params(left=False)

# Título / Subtítulo / Legendas
ax.text(-0.052, 1.17,'Evolução dos óbitos por COVID-19 no Brasil entre 2020 e 2023',
        fontsize=24,
        color=AZUL3,
        fontweight='bold',
        transform=ax.transAxes)

# Subtítulo
ax.text(-0.052, 1.04,'Cerca de 704 mil brasileiros faleceram desde o início da pandemia. '
                    'O pico de óbitos ocorreu durante o primeiro \nsemestre de 2021, '
                    'com dias registrando mais de 4 mil mortes.',
        fontsize=14.5,
        color=AZUL5,
        transform=ax.transAxes)

# Ajustar espaço entre os plots
plt.tight_layout()

# Exibir o gráfico
plt.show() 

